In [2]:
import tensorflow as tf
import os
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import cv2

In [3]:
train_dir = "C:/Users/lenovo/Downloads/ss/simpsons_dataset"
test_dir = "C:/Users/lenovo/Downloads/ss/kaggle_simpson_testset"

In [4]:
train_img = [os.path.join(train_dir, f) for f in os.listdir(train_dir)]
test_img = [os.path.join(test_dir, f) for f in os.listdir(test_dir)]

In [5]:
df_train = pd.read_csv("C:/Users/lenovo/Downloads/ss/number_pic_char.csv")

In [6]:
len(df_train.name.unique())

47

In [7]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
      rescale=1./255,
      rotation_range=25,
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest',
      validation_split = 0.2)

In [10]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(80,80),
                                                    batch_size=150,
                                                    subset = 'training',
                                                    class_mode='categorical',
                                                    color_mode='rgb')

Found 33511 images belonging to 43 classes.


In [11]:
val_datagen=tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split = 0.2)

In [12]:
val_generator = val_datagen.flow_from_directory(train_dir,
                                                target_size=(80,80),
                                                batch_size=150,
                                                subset = 'validation',
                                                class_mode='categorical',
                                                color_mode='rgb')

Found 8355 images belonging to 43 classes.


In [13]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [14]:
test_generator = test_datagen.flow_from_directory(test_dir,
                                                    target_size=(80,80),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    color_mode='rgb')

Found 990 images belonging to 1 classes.


In [15]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(80, 80, 3), activation="relu"))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation="relu"))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Conv2D(256, (3, 3), padding='same', activation="relu")) 
model.add(tf.keras.layers.Conv2D(256, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.Dropout(0.2))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(1024, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(43, activation='softmax'))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 80, 80, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 78, 78, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 39, 39, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 39, 39, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 39, 39, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 37, 37, 64)        36928     
                                                        

In [17]:
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [18]:
callbacks = [
tf.keras.callbacks.EarlyStopping(monitor = 'val_loss',
    min_delta = 0.001,
    mode="max",
    patience = 10,
    verbose = 1,
    restore_best_weights = True),
    
tf.keras.callbacks.ModelCheckpoint(
    filepath='./model.keras',
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)]

In [ ]:
history = model.fit(
    train_generator,
    epochs=10,
    callbacks=callbacks,
    validation_data=val_generator,
    batch_size=300)

Epoch 1/10
224/224 [==============================] - 2225s 10s/step - loss: 2.3300 - accuracy: 0.4964 - val_loss: 2.2456 - val_accuracy: 0.5010
Epoch 2/10
 69/224 [========>.....................] - ETA: 24:35 - loss: 2.2648 - accuracy: 0.4981

In [ ]:
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, "bo", label="Tranning_acc", color = "red")
plt.plot(epochs, val_acc, "b", label="validation_acc", color = "blue")
plt.legend()
plt.show()

In [ ]:
score = model.evaluate(val_generator,batch_size=32)
print("Accuracy: {:.2f}%".format(score[1] * 100)) 
print("Loss: ",score[0])

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
img_path = "/kaggle/input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/bart_simpson_28.jpg"
img = image.load_img(img_path, target_size=(64, 64, 3))
plt.imshow(img)
plt.show()